# NLP : Analyse des sentiments

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
x_train = pd.read_csv("/content/drive/MyDrive/Train.csv")

Mounted at /content/drive


In [ ]:
import re
def supprimer_crochets(text):
  return re.sub('\[[^]]*\]', '', text)
x_train['text']=x_train['text'].apply(supprimer_crochets)
def supprimer_special(text, remove_digits=True):
 pattern=r'[^a-zA-z0-9\s]'
 text=re.sub(pattern,'',text)
 return text

In [ ]:
import nltk
nltk.download(["names","stopwords","punkt"])

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#Stemming the text
def simple_stemmer(text):
 ps=nltk.porter.PorterStemmer()
 text= ' '.join([ps.stem(word) for word in text.split()])
 return text

In [ ]:
#Apply function on review column
x_train['text']=x_train['text'].apply(simple_stemmer)

In [ ]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()
stopword_list=nltk.corpus.stopwords.words('english')

In [ ]:
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
print(stop)

{'your', "you're", 'are', "don't", 'same', 'nor', "needn't", 'had', 'by', 'was', "hadn't", 'during', 'have', 'were', 'too', 'll', 'ain', 'her', 'has', 'more', 'in', 'to', 'out', 'me', 'which', 'doing', 'an', 'with', 'further', 'than', 'them', 'themselves', 'ourselves', 'each', 'been', 'down', 'isn', 'through', 'above', 'these', 'm', "shouldn't", 'i', 'only', 'between', 'ma', 'into', 'of', 'ours', 'because', "she's", 'don', 'itself', 'and', "haven't", 'few', 'not', 'hasn', "you've", 'does', 'their', 'she', 'couldn', 'wouldn', 'about', 'a', 'own', 'for', "you'd", 'you', 'being', 'here', 'myself', "won't", 'herself', 'the', 'both', 'no', 'what', "doesn't", 'yourselves', 'its', 'didn', 'up', "mightn't", 'will', 'weren', 'hadn', 'before', "weren't", 'wasn', 'again', 'as', 'but', 'that', "you'll", 'this', 'him', 'yours', "wasn't", 'd', 'how', 'won', 'just', 'so', 'while', "shan't", 'why', "should've", 'where', 'my', 'such', 'when', 'do', "wouldn't", 's', 'they', 'those', 'under', 'aren', 'ha

In [ ]:
def remove_stopwords(text):
 tokens = tokenizer.tokenize(text)
 tokens = [token.strip() for token in tokens]
 filtered_tokens = [token for token in tokens if token not in stopword_list]
 filtered_text = ' '.join(filtered_tokens)
 return filtered_text
x_train['text']=x_train['text'].apply(remove_stopwords)

In [ ]:
#30000 commentaires pour l'entrainement
norm_train_text=x_train.text[:30000]
norm_train_text[0]

'grew ( b. 1965 ) watch love thunderbirds. mate school watched. play " thunderbirds " befor school , dure lunch school. want virgil scott. one want alan. count 5 becam art form. took children see movi hope would get glimps love child. bitterli disappointing. onli high point wa snappi theme tune. could compar origin score thunderbirds. thank earli saturday morn one televis channel still play rerun seri gerri anderson hi wife created. jonatha frake hand hi director chair , hi version wa complet hopeless. wast film. utter rubbish. cgi remak may accept replac marionett homo sapien subsp. sapien wa huge error judgment .'

In [ ]:
#10000 commentaires pour le test
norm_test_text=x_train.text[30000:]
norm_test_text[35005]

"giorgino long , excruci journey bad wors life protagonist movi named. young demobilized , gas-poison first world war lieuten veri delic health , previous wa doctor orphanag children mental deprivations , goe search new locat find much ever intend quit differ nature. depress atmospher ultim despair , insan one much less horribl sane , mad kind poetry , hold hypnot veri first frame thi film last. beauty : beauti winter mountains , beauti snowi landscap wild woods , ' even know sorrow sad last day war could made beautiful ."

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#Tfidf vectorizer
tv=TfidfVectorizer()
#transformed train reviews
tv_train_text=tv.fit_transform(norm_train_text)
#transformed test reviews
tv_test_text=tv.transform(norm_test_text)
print('Tfidf_train:',tv_train_text.shape)
print('Tfidf_test:',tv_test_text.shape)

Tfidf_train: (30000, 79233)
Tfidf_test: (10000, 79233)


In [ ]:
sentiment_data = x_train['label']

In [ ]:
#Spliting the sentiment data (labels)
train_sentiments=sentiment_data[:30000]
test_sentiments=sentiment_data[30000:]
print(train_sentiments)
print(test_sentiments)

0        0
1        0
2        0
3        0
4        1
        ..
29995    1
29996    1
29997    1
29998    0
29999    1
Name: label, Length: 30000, dtype: int64
30000    0
30001    1
30002    0
30003    0
30004    0
        ..
39995    1
39996    1
39997    0
39998    1
39999    1
Name: label, Length: 10000, dtype: int64


In [ ]:
from sklearn.linear_model import LogisticRegression
#training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1)
#Fitting the model for tfidf features
lr_tfidf=lr.fit(tv_train_text,train_sentiments)
print(lr_tfidf)

LogisticRegression(C=1, max_iter=500)


In [ ]:
#Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(tv_test_text)
print(lr_tfidf_predict)

[0 1 0 ... 0 1 1]


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
lr_tfidf_score=accuracy_score(test_sentiments,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)

lr_tfidf_score : 0.8918


In [ ]:
confusion_matrix(test_sentiments,lr_tfidf_predict)

array([[4397,  575],
       [ 507, 4521]])

In [ ]:
#one comment prediction
liste = []
commentaire = "hello it is a bad movie"
commentaire = supprimer_crochets(commentaire)
commentaire = supprimer_special(commentaire)
commentaire = simple_stemmer(commentaire)
tokens = remove_stopwords(commentaire)
liste.append(tokens)
x = tv.transform(liste)
res = lr.predict(x)
print(res)

[1]


In [ ]:
import joblib

# Save the trained model
joblib.dump(lr_tfidf, 'model/sentiment_model.pkl')

# Save the TfidfVectorizer
joblib.dump(tv, 'model/tfidf_vectorizer.pkl')
